In [3]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Initialize()

In [23]:
ex = np.genfromtxt("RayRoi_status_c.csv", delimiter=',', skip_header=1)
#ex = data[0:5000]
print(ex)

[[29.55412927  3.15692156 29.55637511 ...  0.11569145  0.27797762
          nan]
 [29.55637511  3.15692156 29.55862095 ...  0.12203933  0.28199639
          nan]
 [29.55862095  3.15692156 29.56086678 ...  0.13629541  0.26793651
          nan]
 ...
 [29.63722522  3.06933036 29.63947106 ...  0.10550674  0.26726777
          nan]
 [29.63947106  3.06933036 29.64171689 ...  0.11020866  0.26921628
          nan]
 [29.64171689  3.06933036 29.64396273 ...  0.10611527  0.25816508
          nan]]


In [24]:
new_array = np.empty((0,5), float)
for row in ex:
    min_lon = row[0]
    min_lat = row[1]
    max_lon = row[2]
    max_lat = row[3]
    b5_val = row[13]
    b6_val = row[14]
    #nasa = row[9]
    #elevation = row[10] # GEDI
    #srtm_gedi = row[11] # SRTM - GEDI
    
    # Center Lon/Lat positions:
    b5_b6 = (b5_val - b6_val) / (b5_val + b6_val)
    
    new_row = np.array([min_lon, min_lat, max_lon, max_lat, b5_b6])
    new_array = np.append(new_array, np.array([new_row]), axis=0)
print(new_array)

[[29.55412927  3.15692156 29.55637511  3.15916748 -0.41224006]
 [29.55637511  3.15692156 29.55862095  3.15916748 -0.39589832]
 [29.55862095  3.15692156 29.56086678  3.15916748 -0.32565734]
 ...
 [29.63722522  3.06933036 29.63947106  3.07157629 -0.43393801]
 [29.63947106  3.06933036 29.64171689  3.07157629 -0.41907529]
 [29.64171689  3.06933036 29.64396273  3.07157629 -0.41739778]]


In [27]:
# Remove null values
new_array2 = np.delete(new_array, np.where(new_array[:,4]==-999), axis=0)

In [28]:
# Columns
#lons = new_array2[:,12]
#lats = new_array2[:,13]
#nasa_dem = new_array2[:,9]
# Interpolate all the center lon/lat positions with the elevation data metrics.
#interp = LinearNDInterpolator((lons, lats), nasa_dem)

In [29]:
# test = interp.__call__(24.99, -10.5)
# print(test) # returns nan -> outside of bounds

In [ ]:
new_array3 = np.empty((0,15), float)
for row in new_array:
    # center_lon = 
    # center_lat = row[13]
    # nasa = row[9]
    #elev_gedi = row[10]
    #elev_loss = row[11]
    score = 0
    
    #if (nasa == -999):
     #   nasa = interp.__call__(center_lon, center_lat)
    
    change = 250*0.00001
    current = row[4]
    # Left and right neighbors will have the same center lat
    left = [center_lon - change, center_lat]
    right = [center_lon + change, center_lat]
    # Up and down neighbors will have the same center lon
    up = [center_lon, center_lat + change]
    down = [center_lon, center_lat - change]
    # Corner neighbors
    ul = [center_lon - change, center_lat + change]
    ur = [center_lon + change, center_lat + change]
    dl = [center_lon - change, center_lat - change]
    dr = [center_lon + change, center_lat - change]
    
    neighbors = [left, right, up, down, ul, ur, dl, dr]
    for x in neighbors:
        x_nasa = interp.__call__(x[0], x[1])
        if (nasa < x_nasa):
            score = score+1
            
    new_row = np.append(row, score, axis=None)
    new_array3 = np.append(new_array3, np.array([new_row]), axis=0)

In [ ]:
print(new_array3)

In [ ]:
# Convert to a Feature Collection
# Create a list of Features for areas, with score as property

region_list = []
for row in new_array3:
    region = ee.Geometry.Polygon([[[float(row[0]), float(row[3])],
                                  [float(row[0]), float(row[1])],
                                  [float(row[2]), float(row[1])],
                                  [float(row[2]), float(row[3])]]])
    f = ee.Feature(region).set('elevation score', float(row[14]))
    region_list.append(f)

# Wrap feature list in a Feature Collection
fc = ee.FeatureCollection(region_list)

# Export the Feature Collection to Google Earth Engine (GEE)
task = ee.batch.Export.table.toAsset(**{
  'collection': fc,
  'description':'compiled_results',
  'assetId': 'users/EmilyNason/compiledResultsNasaDemLower', # must be unique name
});

task.start()